In [46]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [52]:
#!unzip "drive/MyDrive/Colab Notebooks/datasets/kagglecatsanddogs_3367a.zip" -d "drive/MyDrive/Colab Notebooks/datasets/" -y

Archive:  drive/MyDrive/Colab Notebooks/datasets/kagglecatsanddogs_3367a.zip
caution: filename not matched:  -y


In [58]:
import cv2, os
import numpy as np
from tqdm import tqdm

REBUILD_DATA = False

class DogsVCats():
    img_size = 50
    cats = "drive/MyDrive/Colab Notebooks/datasets/PetImages/Cat"
    dogs = "drive/MyDrive/Colab Notebooks/datasets/PetImages/Dog"
    labels = {cats: 0, dogs: 1}

    training_data = []
    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.labels:
            print(label)
            for image in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, image)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.img_size, self.img_size))
                    self.training_data.append([np.array(img), 
                                              np.eye(2)[self.labels[label]]])
                    
                    if label == self.cats:
                      self.catcount += 1
                    elif label == self.dogs:
                      self.dogcount += 1
                
                except Exception as e:
                    pass
        
        np.random.shuffle(self.training_data)
        np.save("drive/MyDrive/Colab Notebooks/datasets/training_data.npy", self.training_data)
        print("Cats: ", self.catcount)
        print("Dogs: ", self.dogcount)

if REBUILD_DATA:
    dogsvcats = DogsVCats()
    dogsvcats.make_training_data()


  0%|          | 0/12501 [00:00<?, ?it/s]

drive/MyDrive/Colab Notebooks/datasets/PetImages/Cat


  0%|          | 0/12501 [00:00<?, ?it/s]

drive/MyDrive/Colab Notebooks/datasets/PetImages/Dog


100%|██████████| 12501/12501 [00:51<00:00, 242.28it/s]


Cats:  12476
Dogs:  12470


In [57]:
training_data = np.load("training_data.npy", allow_pickle=True)

In [59]:
print(len(training_data))
print(training_data.shape)

24946
(24946, 2)


In [60]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5) #Input, Output, Kernel size
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)

        self.fc1 = nn.Linear(128*8*8,512)
        self.fc2 = nn.Linear(512,2)

        #x = torch.randn(50,50).view(-1,1,50,50)
        #self._to_linear = None

    def forward(self, x):
        x = self.conv1(x) #Output is (32 x 46 x 46)
        x = F.relu(x)
        x = self.conv2(x) #Output is (64 x 42 x 42)
        x = F.relu(x)
        x = F.max_pool2d(x,2) #Output is (64 x 21 x 21)
        x = self.conv3(x)     #Output is (128 x 17 x 17)
        x = F.relu(x)
        x = F.max_pool2d(x,2) #Output is (128 x 8 x 8)
        x = torch.flatten(x,1) 
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

net = Net()

In [62]:
optimizer = optim.Adam(net.parameters(), lr= 0.001)
loss_function = nn.MSELoss()

X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255.0
y = torch.Tensor([i[1] for i in training_data])

Val_Pct = 0.1
val_size = int(len(X)*Val_Pct)
print(val_size)

2494


In [63]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_X))

22452
2494


In [65]:
batch_size = 100
EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X),batch_size)):
        #print(i, i+batch_size) 
        batch_X = train_X[i:i+batch_size].view(-1,1,50,50)
        batch_y = train_y[i:i+batch_size]

        net.zero_grad()
        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()


100%|██████████| 225/225 [06:55<00:00,  1.85s/it]

tensor(1.6711, grad_fn=<MseLossBackward>)


In [66]:
print(loss)

tensor(1.6711, grad_fn=<MseLossBackward>)


In [67]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1,1,50,50))[0]

        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct += 1
        total += 1

print("Accuracy:", round(correct/total,3))


100%|██████████| 2494/2494 [00:20<00:00, 120.85it/s]

Accuracy: 0.583
